In [2]:
#!pip install --user nba_api
#!pip install --user pandas
#!pip install --user requests
import sys  
!{sys.executable} -m pip install --user nba_api

In [3]:
headers  = {
    'Connection': 'keep-alive',
    'Accept': 'application/json, text/plain, */*',
    'x-nba-stats-token': 'true',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36',
    'x-nba-stats-origin': 'stats',
    'Sec-Fetch-Site': 'same-origin',
    'Sec-Fetch-Mode': 'cors',
    'Referer': 'https://stats.nba.com/',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'en-US,en;q=0.9',
}

In [4]:



import pandas as pd
import requests
import time
from IPython.display import display, HTML
from nba_api.stats.static import teams

from nba_api.stats.static import players
from nba_api.stats.endpoints import commonplayerinfo
from nba_api.stats.endpoints import leaguegamefinder
from nba_api.stats.endpoints import playbyplayv2

pd.options.display.max_columns = None
pd.options.display.max_rows = None


In [5]:
#Get information about playyers

#nba_players = players.get_players()
#df = pd.DataFrame(nba_players)
#df.sample(5)



active_players = players.get_active_players()
df = pd.DataFrame(active_players)
df.sample(10)

active_players = players.get_active_players()
df = pd.DataFrame(active_players)

booker = df.loc[(df["last_name"] == "Booker")]
booker

,id,full_name,first_name,last_name,is_active
50,1626164,Devin Booker,Devin,Booker,True


In [6]:
#Get Team Information
nba_teams = teams.get_teams()
df = pd.DataFrame(nba_teams)


suns = teams.find_team_by_abbreviation('phx')
#suns


# get game logs from the reg season
gamefinder = leaguegamefinder.LeagueGameFinder(season_nullable='2022-23',league_id_nullable='00',team_id_nullable="1610612756",season_type_nullable='Regular Season')
#print(type(gamefinder))
games = gamefinder.get_data_frames()[0]
game_ids = games["GAME_ID"].to_list()

f = open("demofile3.txt", "w")

#for game in game_ids:
#    pbp = playbyplayv2.PlayByPlayV2(game)
#    pbp = pbp.get_data_frames()[0]
#    f.write(pbp.to_string())
#    time.sleep(1)
f.close()
print("yo")

yo


In [27]:
def logRuns(pbp, team):
    f = open("demofile3.txt", "w")
    timeout = (team.upper() + " Timeout")
    
    btat = []
    pbpf = pbp.loc[(pbp['SCORE'].notna()) |  (pbp['HOMEDESCRIPTION'].str.contains(team.upper() + " Timeout")) ]
    pbpf = pbpf.reset_index(drop=True)
    f.write(pbpf.to_string())
    f.close()
    
    
    for index,play in pbpf.iterrows():
        playHD = str(play["HOMEDESCRIPTION"]).upper()
        if timeout.upper() in playHD:
#            print(str(index))
            #was there a run?
            
            if index > 10:
                startscore = pbpf.iloc[index -11]['SCORE'].split(" - ")
                endscore = pbpf.iloc[index -1]['SCORE'].split(" - ")
                
                homediff = int(endscore[1])-int(startscore[1])
                visitdiff = int(endscore[0])-int(startscore[0])
                bt_diff = visitdiff -homediff
                
 #               print(str(visitdiff -homediff))
 #               print(startscore)
 #               print(endscore)
                
                if index + 10 < len(pbpf) and bt_diff >= 7:
                    #print(str(visitdiff -homediff))
                    #print(startscore)
                    #print(endscore)
                    startscore = pbpf.iloc[index - 1]['SCORE'].split(" - ")
                    endscore = pbpf.iloc[index + 10]['SCORE'].split(" - ")

                    homediff = int(endscore[1])-int(startscore[1])
                    visitdiff = int(endscore[0])-int(startscore[0])
                    at_diff = visitdiff -homediff

                    #print(str(visitdiff -homediff))
                    #print(startscore)
                    #print(endscore)
                    btat.append([bt_diff, at_diff])

                

                        
                
    return btat


game = game_ids[81]
pbp = playbyplayv2.PlayByPlayV2(game)
pbp = pbp.get_data_frames()[0]

print("[Before timeout, After timeout]")
print(logRuns(pbp, suns["nickname"]))

[Before timeout, After timeout]
[[7, 2], [10, 2], [14, -12]]


In [11]:

display(pbp.loc[1:2])

,GAME_ID,EVENTNUM,EVENTMSGTYPE,EVENTMSGACTIONTYPE,PERIOD,WCTIMESTRING,PCTIMESTRING,HOMEDESCRIPTION,NEUTRALDESCRIPTION,VISITORDESCRIPTION,SCORE,SCOREMARGIN,PERSON1TYPE,PLAYER1_ID,PLAYER1_NAME,PLAYER1_TEAM_ID,PLAYER1_TEAM_CITY,PLAYER1_TEAM_NICKNAME,PLAYER1_TEAM_ABBREVIATION,PERSON2TYPE,PLAYER2_ID,PLAYER2_NAME,PLAYER2_TEAM_ID,PLAYER2_TEAM_CITY,PLAYER2_TEAM_NICKNAME,PLAYER2_TEAM_ABBREVIATION,PERSON3TYPE,PLAYER3_ID,PLAYER3_NAME,PLAYER3_TEAM_ID,PLAYER3_TEAM_CITY,PLAYER3_TEAM_NICKNAME,PLAYER3_TEAM_ABBREVIATION,VIDEO_AVAILABLE_FLAG
1,0022200013,4,10,0,1,10:11 PM,12:00,Jump Ball Ayton vs. McGee: Tip to Paul,None,None,None,None,4,1629028,Deandre Ayton,1.610613e+09,Phoenix,Suns,PHX,5,201580,JaVale McGee,1.610613e+09,Dallas,Mavericks,DAL,4,101108,Chris Paul,1.610613e+09,Phoenix,Suns,PHX,1
2,0022200013,10,7,5,1,10:11 PM,11:49,None,None,McGee Violation:Kicked Ball (E.Lewis),None,None,5,201580,JaVale McGee,1.610613e+09,Dallas,Mavericks,DAL,0,0,None,NaN,None,None,None,1,0,None,NaN,None,None,None,0
